# Diagnóstico de Aplicação com LLM

In [1]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Carregar o modelo DeepSeek R1 Distill Llama em 8-bit
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Quantização 8-bit
    device_map="auto"   # Distribui o modelo automaticamente entre CPU/GPU
)

2025-02-25 13:24:00.326813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740500640.348577   63537 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740500640.353915   63537 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 13:24:00.376785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `Bi

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# Faça 3 consultas no prometheus para as seguintes métricas: requests_total, errors_total e latency

# Importe a biblioteca prometheus_client
from prometheus_client import start_http_server, Summary, Counter
from prometheus_client.core import REGISTRY
import requests
import json

# Faça a requisição para o servidor Prometheus
def query_prometheus(query):
    response = requests.get("http://localhost:30090/api/v1/query", params={"query": query})
    data = json.loads(response.text)
    return data["data"]["result"]


def get_prometheus_metrics():
    # Consulta para a métrica requests_total com a média dos últimos 5 minutos
    requests_total = query_prometheus("avg(rate(requests_total[5m]))*60")
    requests_total = round(float(requests_total[0]["value"][1]), 2)

    # Consulta para a métrica errors_total com a média dos últimos 5 minutos
    errors_total = query_prometheus("avg(rate(errors_total[5m]))*60")
    errors_total = round(float(errors_total[0]["value"][1]), 2)

    # Consulta para a métrica latency com a média dos últimos 5 minutos
    latency = query_prometheus("rate(latency_milliseconds_sum[5m])")
    latency = round(float(latency[0]["value"][1]), 2)

    return requests_total, errors_total, latency

In [3]:
# Testando a requisição no Prometheus

requests_total, errors_total, latency = get_prometheus_metrics()

texto = f"Latência: {latency} segundos, trafego: {requests_total}, taxa de erros: {errors_total}%"
print(texto)

Latência: 5.3 segundos, trafego: 53.27, taxa de erros: 4.63%


In [4]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


class GPTJLocal(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Exemplos de few-shot learning para melhorar a qualidade das respostas
        few_shot_examples = """
        Você é um assistente de AIOps. Sua tarefa é analisar as métricas de desempenho de uma aplicação, comparar as métricas atuais com o histórico e responder à pergunta de forma clara e precisa. Siga os seguintes passos:
        1. Analise cada métrica individualmente.
        2. Compare com o que é esperado.
        3. Identifique problemas ou anomalias.
        4. Conclua com uma avaliação geral.

        Métricas normais para esta aplicação (SLO):
        - Latência: Menor que 10 segundos
        - Taxa de erros: Menor que 20%
        - Trafego: Menos de 100 requisições por segundo

        Exemplo 1:
        Métricas atuais: Latência: 20.39 segundos, trafego: 48.63, taxa de erros: 6.74%
        Pergunta: Avalie minha aplicação. Está tudo bem?
        Resposta:
        Conclusão: A aplicação não está saudável devido à alta latência. Recomendo investigar a causa do aumento.

        Exemplo 2:
        Métricas atuais: Latência: 4.39 segundos, trafego: 48.63, taxa de erros: 6.74%
        Pergunta: Avalie minha aplicação. Está tudo bem?
        Resposta:
        Conclusão: A aplicação está saudável e com desempenho dentro do esperado.

        Exemplo 3:
        Métricas atuais: Latência: 40.39 segundos, trafego: 480.63, taxa de erros: 6.74%
        Pergunta: Avalie minha aplicação. Está tudo bem?
        Resposta:
        Conclusão: A aplicação não está saudável devido a alta latencia. Provavelmente ocorrendo devido ao aumento do trafego.

        """

        # Combinar os exemplos de few-shot com o prompt do usuário
        full_prompt = few_shot_examples + f"\n\nMétricas atuais: {prompt}\nResposta:"

        # Tokenizar a entrada
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        
        # Gerar texto com parâmetros ajustados
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=1024,
                temperature=0.2,  # Controla a criatividade (valores mais baixos = mais determinístico)
                top_p=0.8,        # Amostragem nucleus (controla a diversidade)
                top_k=40,         # Limita o vocabulário às top-k palavras
                do_sample=True,     # Ativa a amostragem
                num_beams=3,       # Geração de feixe para melhorar a coerência
                early_stopping=True,          # Para a geração quando a resposta estiver completa
                pad_token_id=tokenizer.eos_token_id  # Usa o token de fim de texto para parar
            )
        
        # Decodificar a saída
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extrair apenas a resposta gerada (ignorar o prompt)
        response = output_text.split("Resposta:")[-1].strip()

        # Verificar se a resposta está completa
        if "Conclusão:" not in response:
            response += "\nConclusão: A aplicação precisa de uma análise mais detalhada."

        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "DeepSeek-R1-Distill-Llama-8B"}

    @property
    def _llm_type(self) -> str:
        return "Llama"

# Instanciar o modelo local
local_llm = GPTJLocal()

# Definir o prompt
template = """
Você é um assistente de AIOps. Analise as métricas de desempenho de uma aplicação abaixo, compare as atuais com o histórico e responda à pergunta. Siga os seguintes passos:
1. Analise cada métrica individualmente.
2. Compare com o que é esperado.
3. Identifique diferenças nas métricas, problemas ou anomalias.
4. Conclua com uma avaliação geral.

Métricas atuais: {texto}
Pergunta: {question}
"""
prompt = PromptTemplate(template=template, input_variables=["texto", "question"])

# Criar a cadeia
chain = LLMChain(llm=local_llm, prompt=prompt)

# Função para interpretar o estado da aplicação
def interpret_application_state(texto, question):
    response = chain.run(texto=texto, question=question)
    return response

/tmp/ipykernel_63537/2825321465.py:101: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=local_llm, prompt=prompt)


### Situação de Métricas Normais

In [5]:
requests_total, errors_total, latency = get_prometheus_metrics()

texto = f"Latência: {latency} segundos, trafego: {requests_total}, taxa de erros: {errors_total}%"
print(texto)
question = "Avalie minha aplicação. Está tudo bem?"

# Interpretar o estado da aplicação
response = interpret_application_state(texto, question)
print(response)

Latência: 4.3 segundos, trafego: 50.32, taxa de erros: 5.47%


/tmp/ipykernel_63537/2825321465.py:105: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(texto=texto, question=question)


Conclusão: A aplicação está saudável e com desempenho dentro do esperado.
</think>

Conclusão: A aplicação está saudável e com desempenho dentro do esperado.


### Elevando a taxa de erros

In [6]:
requests_total, errors_total, latency = get_prometheus_metrics()

# Multiplicando os erros por 5
errors_total = errors_total * 5

texto = f"Latência: {latency} segundos, trafego: {requests_total}, taxa de erros: {errors_total}%"
print(texto)
question = "Faça um diagnóstico da minha aplicação."

# Interpretar o estado da aplicação
response = interpret_application_state(texto, question)
print(response)

Latência: 4.63 segundos, trafego: 49.9, taxa de erros: 41.050000000000004%
**

Conclusão: Apesar da latência e trafego estarem dentro dos limites, a taxa de erros elevada é um sinal de alerta. Precisamos investigar o que está causando o


## Elevando o tráfego

In [7]:
requests_total, errors_total, latency = get_prometheus_metrics()

# Multiplicando o tráfego  por 4
requests_total = requests_total * 4

texto = f"Latência: {latency} segundos, trafego: {requests_total}, taxa de erros: {errors_total}%"
print(texto)
question = "Como está minha aplicação?"

# Interpretar o estado da aplicação
response = interpret_application_state(texto, question)
print(response)

Latência: 5.63 segundos, trafego: 207.16, taxa de erros: 8.0%
Conclusão: A aplicação está saudável e com desempenho dentro do


### Próximos passos

- Buscar métricas de banco de dados
- Buscar métricas do servidor de aplicação
- Tentar identificar se o problema é no banco de dados ou na aplicação